# The Brain class

The Colliseum class is in charged of 

In [8]:
import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from IPython.display import Image
%matplotlib qt

import sys
sys.path.insert(0, "..")  # Adds higher directory to python modules path.

from traphing.data_classes import Velas, Portfolio
from traphing import Brain
from traphing.strategies import Trade, Coliseum
from traphing.strategies.entry import EntryStrategy, CrossingMovingAverages
from traphing.strategies.exit import ExitStrategy, StopLoss

from traphing.utils import Timeframes, unwrap
import traphing.utils  as ul

from traphing.graph.Gl import gl

folder_images = "./images/python_generated/strategies/"

## Setting up a portfolio

Strategies work on porfolio objects, they are part of them.


In [9]:
symbol_names_list = ["AUDCHF"]
timeframes_list = [Timeframes.M15]
portfolio_name = "my_portfolio"

storage_folder = "../tests/data/storage/"

portfolio = Portfolio(portfolio_name, symbol_names_list, timeframes_list)
portfolio.load_data_from_csv(storage_folder)

start_time = dt.datetime(2019,7,20); end_time = dt.datetime(2019,7,25)
portfolio.set_time_interval(start_time,end_time)

Size ../tests/data/storage/M15/AUDCHF_M15.csv:  100400  rows


## Create a Coliseum object with two entry trading strategies

In [10]:
timeframe = timeframes_list[0]
symbol_name = symbol_names_list[0]
# First entry strategy
entry_strategy1 = CrossingMovingAverages("Crossing averages 1", portfolio)
symbol_name = symbol_names_list[0]
slow_MA_params = {"symbol_name":symbol_name,"timeframe": timeframe,"indicator_name":"SMA", "args": {"n":45}}
fast_MA_params = {"symbol_name":symbol_name,"timeframe": timeframe,"indicator_name":"SMA", "args":{"n":20}}
entry_strategy1.set_slow_MA(slow_MA_params)
entry_strategy1.set_fast_MA(fast_MA_params)

# Second entry strategy
entry_strategy2 = CrossingMovingAverages("Crossing averages 2", portfolio)
slow_MA_params = {"symbol_name":symbol_name,"timeframe": timeframe,"indicator_name":"SMA", "args": {"n":50}}
fast_MA_params = {"symbol_name":symbol_name,"timeframe": timeframe,"indicator_name":"SMA", "args":{"n":10}}
entry_strategy2.set_slow_MA(slow_MA_params)
entry_strategy2.set_fast_MA(fast_MA_params)

coliseum = Coliseum()
coliseum.add_entry_strategy(entry_strategy1)
coliseum.add_entry_strategy(entry_strategy2)

# The Brain Class

The Coliseum Class contains all the Entry and Exit strategies.

In [11]:
brain = Brain(coliseum, portfolio)

In [12]:
brain.backtest()

In [13]:
brain.coliseum.compute_requests_queue().qsize()

12

In [ ]:
image_name = "entry_and_exit.png"; img_path = folder_images + image_name
gl.init_figure()

n_strategies = len(coliseum.entry_strategies_ids_list)

n_rows, n_cols = n_strategies*2,1; size_inches = [12, 8]
ax = None; axes_list = []
for i in range(n_strategies*2):
    ax = gl.subplot2grid((n_rows, n_cols),(i,0), sharex = ax)
    axes_list.append(ax)
    
i = 0
for strategy_id in coliseum.entry_strategies_ids_list:
    ax1 = axes_list[i*2]; ax2 = axes_list[i*2+1]
    entry_strategy = coliseum.get_entry_strategy(strategy_id)
    entry_strategy_series = entry_strategy.compute_strategy_series()
    entry_series = entry_strategy.compute_entry_series()

    portfolio[symbol_name][timeframe].plot_barchart(axes = ax1, labels = ["", "", strategy_id])
    gl.plot(entry_strategy_series.index, entry_strategy_series, legend = list(entry_strategy_series.columns), axes =ax1)

    gl.stem(entry_strategy_series.index,entry_series, axes = ax2, legend = "Trades")

    i = i+1
gl.subplots_adjust(left=.09, bottom=.10, right=.90, top=.95, wspace=.20, hspace=0, hide_xaxis = True)

gl.save_figure(img_path, dpi = 100, size_inches = size_inches, close = True)
Image(img_path)